In [10]:
import warnings
warnings.filterwarnings("ignore")


import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv('data/istanbul_data_fully_preprocessed.csv')

data.info()

name_cols = ['MainDistID', 'DistID', 'HcoID', 'SkuID', 'BrandID', 'District']
categorical_cols = ['HcoType', 'SalesRegionID'] 
numerical_cols = ['PaidQty', 'FGQty', 'TotalQuantity', 'Week', 'Quarter', 'Month', 'Year']
boolean_cols = ['IsReturn', 'IsMrsOrder', 'Status']

# convert categorical columns to 'category' data type
for col in categorical_cols:
    data[col] = data[col].astype('category')

# convert numerical columns to 'int' data type
for col in numerical_cols:
    data[col] = data[col].astype('int')
    
# convert name columns to str data type
for col in name_cols:
    data[col] = data[col].astype(str)

# convert boolean columns to 'bool' data type
for col in boolean_cols:
    data[col] = data[col].astype(bool)
    
raw_data = data.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 863002 entries, 0 to 863001
Data columns (total 19 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   MainDistID     863002 non-null  int64 
 1   DistID         863002 non-null  int64 
 2   Date           863002 non-null  object
 3   HcoID          863002 non-null  int64 
 4   BrandID        863002 non-null  int64 
 5   SkuID          863002 non-null  int64 
 6   IsReturn       863002 non-null  bool  
 7   PaidQty        863002 non-null  int64 
 8   FGQty          863002 non-null  int64 
 9   IsMrsOrder     863002 non-null  bool  
 10  HcoType        863002 non-null  object
 11  District       857497 non-null  object
 12  Status         863002 non-null  bool  
 13  TotalQuantity  863002 non-null  int64 
 14  Week           863002 non-null  int64 
 15  Month          863002 non-null  int64 
 16  Quarter        863002 non-null  int64 
 17  Year           863002 non-null  int64 
 18  Sale

In [5]:
segment_1_sku_ids = ["568", "578", "1050"]
segment_2_sku_ids = ["319", "509", "615", "1018"]

all_sku_ids = segment_1_sku_ids + segment_2_sku_ids

data = data[data['SkuID'].isin(all_sku_ids)]

In [6]:
quarterly_data = data.copy()

quarterly_data = quarterly_data.groupby(['Year', 'Quarter', 'SkuID']).agg({'TotalQuantity': 'sum'}).reset_index()
quarterly_data

,Year,Quarter,SkuID,TotalQuantity
0,2019,1,1018,13937
1,2019,1,1050,52981
2,2019,1,319,46823
3,2019,1,509,43084
4,2019,1,568,145879
...,...,...,...,...
79,2021,4,319,76375
80,2021,4,509,36852
81,2021,4,568,225966
82,2021,4,578,109939


In [7]:
sku_319 = quarterly_data[quarterly_data['SkuID'] == "319"]
sku_319

,Year,Quarter,SkuID,TotalQuantity
2,2019,1,319,46823
9,2019,2,319,42588
16,2019,3,319,42818
23,2019,4,319,49943
30,2020,1,319,56466
37,2020,2,319,49173
44,2020,3,319,50841
51,2020,4,319,59334
58,2021,1,319,62174
65,2021,2,319,53335


In [8]:
import itertools
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

# Parametre kombinasyonlarını oluştur
p_values = range(0, 3) # Örnek olarak 0'dan 3'e kadar p değerleri
d_values = range(0, 2) # Örnek olarak 0'dan 2'ye kadar d değerleri
q_values = range(0, 3) # Örnek olarak 0'dan 3'e kadar q değerleri
param_combinations = list(itertools.product(p_values, d_values, q_values))

best_score, best_params, best_model, mape = float("inf"), None, None, None

df = sku_319.copy()
df['Date'] = pd.to_datetime(df['Year'].astype(str) + '-' + df['Quarter'].astype(str))
df.set_index('Date', inplace=True)
df.index = df.index.to_period('Q').to_timestamp('S')

# Grid search
for param in param_combinations:
    try:
        model = ARIMA(df['TotalQuantity'], order=param)
        model_fit = model.fit()
        y_pred = model_fit.forecast(steps=2)  
        mse = mean_squared_error(df['TotalQuantity'].iloc[-2:], y_pred)
        mape = mse / df['TotalQuantity'].iloc[-2:].mean()
        if mse < best_score:
            best_score, best_params, best_model, mape = mse, param, model_fit, mape
    except:
        continue

print('Best ARIMA Model:', best_params)
print('MSE:', best_score)
print('MAPE:', mape)
print(best_model.summary())

/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `sta

Best ARIMA Model: (0, 1, 1)
MSE: 232777372.57275793
MAPE: 4176.388632966623
                               SARIMAX Results                                
Dep. Variable:          TotalQuantity   No. Observations:                   12
Model:                 ARIMA(0, 1, 1)   Log Likelihood                -116.039
Date:                Sun, 02 Jun 2024   AIC                            236.078
Time:                        15:34:39   BIC                            236.874
Sample:                             0   HQIC                           235.576
                                 - 12                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1         -0.1204      0.845     -0.142      0.887      -1.777       1.536
sigma2      1.135e+08   9.25e-09   1.23e+16      0.000 

/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [9]:
import itertools
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

dfSarimax = sku_319.copy()
df['Date'] = pd.to_datetime(df['Year'].astype(str) + '-' + df['Quarter'].astype(str))
df.set_index('Date', inplace=True)

# Parametre kombinasyonlarını oluştur
p_values = range(0, 3) # Örnek olarak 0'dan 3'e kadar p değerleri
d_values = range(0, 2) # Örnek olarak 0'dan 2'ye kadar d değerleri
q_values = range(0, 3) # Örnek olarak 0'dan 3'e kadar q değerleri
P_values = range(0, 3) # Örnek olarak 0'dan 3'e kadar P değerleri
D_values = range(0, 2) # Örnek olarak 0'dan 2'ye kadar D değerleri
Q_values = range(0, 3) # Örnek olarak 0'dan 3'e kadar Q değerleri
s_values = range(4, 5) # Örnek olarak 4'den 5'e kadar s değerleri
param_combinations = list(itertools.product(p_values, d_values, q_values, P_values, D_values, Q_values, s_values))

best_score, best_params, best_model, mape = float("inf"), None, None, None

# Grid search

for param in param_combinations:
    try:
        model = SARIMAX(dfSarimax['TotalQuantity'], order=param[:3], seasonal_order=param[3:])
        model_fit = model.fit()
        y_pred = model_fit.forecast(steps=2)
        mse = mean_squared_error(dfSarimax['TotalQuantity'].iloc[-2:], y_pred)
        mape = mse / dfSarimax['TotalQuantity'].iloc[-2:].mean()
        if mse < best_score:
            best_score, best_params, best_model, mape = mse, param, model_fit, mape
    except:
        continue
        
print('Best SARIMA Model:', best_params)
print('MSE:', best_score)
print('MAPE:', mape)
print(best_model.summary())

/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
 This problem is unconstrained.
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/ts

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            1     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.23201D+01    |proj g|=  0.00000D+00

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    1      0      1      0     0     0   0.000D+00   1.232D+01
  F =   12.320109562320384     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            2     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.9

 This problem is unconstrained.


/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
 This problem is unconstrained.
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/senaoz/anaconda3/lib/python3.11/site-

At iterate   10    f=  1.10114D+01    |proj g|=  8.74550D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    4     14     15      1     0     0   3.003D-05   1.101D+01
  F =   11.010734794810999     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            2     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.17393D+01    |proj g|=  1.78677D+00

At iterate    5    f=  6.70403D+00    |proj g|=  4.67040D-06

           * * *

Tit   = total number of iterations
Tnf   = total num

...

/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  5.94014D+00    |proj g|=  3.87982D-01

At iterate    5    f=  5.86734D+00    |proj g|=  1.49363D-01

At iterate   10    f=  5.84316D+00    |proj g|=  8.50053D-02

At iterate   15    f=  5.84180D+00    |proj g|=  5.75011D-03

At iterate   20    f=  5.84133D+00    |proj g|=  3.63471D-02

At iterate   25    f=  5.84080D+00    |proj g|=  1.37935D-03

At iterate   30    f=  5.84076D+00    |proj g|=  1.44989D-02

At iterate   35    f=  5.83273D+00    |proj g|=  3.39459D-01

At iterate   40    f=  5.76861D+00    |proj g|=  3.61132D-01

At iterate   45    f=  5.73160D+00    |proj g|=  1.76000D-01


/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
 This problem is unconstrained.



At iterate   50    f=  5.72701D+00    |proj g|=  8.42715D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    8     50     66      1     0     0   8.427D-03   5.727D+00
  F =   5.7270146316635602     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            9     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  5.94014D+00    |proj g|=  3.87982D-01

At iterate    5    f=  5.87323D+00    |proj g|=  6.83203D-01

At iterate   10    f=  5.84122D+00    |proj g|=  2.28239D-02

At iter

/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
